In [1]:
from IPython.display import Image, clear_output, display
from ipywidgets import interact, widgets

from bo.plots import visualize_improvement

## Finding improvement in Bayesian Optimization

In [2]:
visualize_improvement("poi")

  0%|          | 0/500 [00:00<?, ?it/s]

/Users/alextanhongpin/Library/Caches/pypoetry/virtualenvs/python-bayesian-optimization-in-action-aU6qUxK9-py3.12/lib/python3.12/site-packages/botorch/optim/optimize.py:652: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL: .')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

(tensor([[1.0000],
         [2.0000],
         [0.5985],
         [1.0036],
         [1.0238],
         [1.0789],
         [1.1452],
         [1.2291],
         [1.3242],
         [1.4204],
         [1.4916],
         [1.5247]]),
 tensor([1.6054, 1.5029, 1.0283, 1.6114, 1.6447, 1.7349, 1.8396, 1.9618, 2.0782,
         2.1620, 2.1964, 2.2031]))

In [3]:
play = widgets.Play(
    value=0,
    min=0,
    max=9,
    step=1,
    interval=500,
    description="Press play",
    disabled=False,
)
slider = widgets.IntSlider(min=0, max=9)
widgets.jslink((play, "value"), (slider, "value"))
widgets.HBox([play, slider])


@interact(play=play, slider=slider)
def f(play, slider):
    clear_output(wait=True)
    display(Image(f"tmp/poi_{play}.png"))

interactive(children=(Play(value=0, description='Press play', interval=500, max=9), IntSlider(value=0, descrip…

## Optimizing the expected value of improvement

In [4]:
visualize_improvement("ei")

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

/Users/alextanhongpin/Library/Caches/pypoetry/virtualenvs/python-bayesian-optimization-in-action-aU6qUxK9-py3.12/lib/python3.12/site-packages/botorch/optim/optimize.py:652: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL: .')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)


  0%|          | 0/500 [00:00<?, ?it/s]

(tensor([[ 1.0000],
         [ 2.0000],
         [-0.6873],
         [ 3.4936],
         [-2.2586],
         [ 1.4894],
         [-3.3334],
         [-1.5317],
         [-5.0000],
         [-5.0000],
         [ 5.0000],
         [ 4.7933]]),
 tensor([ 1.6054,  1.5029,  0.9886, -0.7110,  1.1852,  2.1957, -0.0878,  1.0494,
          4.1659,  4.1659,  4.8633,  6.5738]))

In [5]:
play = widgets.Play(
    value=0,
    min=0,
    max=9,
    step=1,
    interval=500,
    description="Press play",
    disabled=False,
)
slider = widgets.IntSlider(min=0, max=9)
widgets.jslink((play, "value"), (slider, "value"))
widgets.HBox([play, slider])


@interact(play=play, slider=slider)
def f(play, slider):
    clear_output(wait=True)
    display(Image(f"tmp/ei_{play}.png"))

interactive(children=(Play(value=0, description='Press play', interval=500, max=9), IntSlider(value=0, descrip…